In [1]:
# Demonstration notebook for RGA analysi for WEST

# standard Python packages
import scipy as sp
import os, glob
import matplotlib.pyplot as plt

# RGA data handling code
# main module
from rga_data_handling import Class as rga
# plotting functions
from rga_data_handling import RGA_class_plotter as plot
# evaluation of impact of uncertainty of cracking patterns
from rga_data_handling import fit_many_times as fmt

# RGA data reader
from WEST_RGA_import import read_MID
# WEST pulse times
from WEST_logbook import WEST_logbook

local_path = os.environ['LOCAL_DATA_PATH']

# location of data files
maindir = os.path.join(local_path, 'WEST', 'RGA','SP1','2018_12')
# location of the logbook file
logbook_dir = os.path.join(local_path, 'WEST','logbook')
# logbook filename
logbook_fn = 'West_pulse_times.txt'

# initialize a logbook object instance
logbook = WEST_logbook(os.path.join(logbook_dir, logbook_fn))

list_of_RGA_files = glob.glob(os.path.join(maindir, "*.asc"))

In [14]:
# read the RGA recordings

# file name dictionary
# one file for each day

fn_d = {'2018-12-18': 'C:\\Pest\\WEST\\RGA\\SP1\\2018_12\\2018-12-18_ammonia6.asc',
       '2018-12-19': 'C:\\Pest\\WEST\\RGA\\SP1\\2018_12\\2018-12-18_ammonia8.1.asc'}

# data dictionary
# one dictionary entry for each date
col_d = {}
for day in fn_d.keys():
    # data is returned as dictionary
    # columns contains the time column, columns['hours'], and intensities at masses, e.g. columns[28]
    err, tag, columns = read_MID(fn_d[day])
    # tag is the metadata
    col_d[day] = {'tag': tag, 'columns': columns}

In [16]:
# time of pre
pre_s = 32

def trace_at_shot(shot, length = 300):
    """Args
        shot
        optional: length - length of recording in s, default = 300
        
        Returns:
        rga_data_handling Trace object"""
    date, time = logbook.shot_date(shot)
    hours = logbook.shot_hours(shot)
    time_col = 'hours'
    tag = col_d[date]['tag']
    columns = col_d[date]['columns']
    
    window = [hours, hours + float(length)/3600]
    new_columns = plot.focus_columns(columns, window, time_col_name=time_col)
    try:
        new_columns['time'] -= new_columns['time'][0]
        new_columns['time'] -= pre_s
    except IndexError:
        tr = rga.Trace({}, {'title': 'empty trace'})
        return tr
    tr = rga.Trace(new_columns, tag)
    tr.tag['shot'] = shot
    tr.tag['title'] = shot
    tr.set_timecol('time')
    # load the cracking patterns from the calibration file 'WEST_CP.txt'
    # the calibration file is in the same directory as the notebook
    tr.replace_CP('WEST_CP.txt')
    return tr

In [13]:
# WEST shot times logbook

print 'All the shots on 18.12.2018'
print logbook.shots_at_date('2018-12-18')
print ''
print 'Date and time of shot 54143'
print logbook.shot_date(54143)
print ''
print 'Closest shot to 18:00, on 18.12.2018 - note, shot time is given and returned as float (hours)'
print logbook.shot_at_hours('2018-12-18', 18.0)
print ''
print 'Time of shot 54143, as float'
print logbook.shot_hours(54143)

All the shots on 18.12.2018
[54117 54118 54119 54120 54121 54122 54123 54124 54125 54126 54127 54128
 54129 54130 54131 54132 54133 54134 54135 54136 54137 54138 54139 54140
 54141 54142 54143 54144 54145 54117 54118 54119 54120 54121 54122 54123
 54124 54125 54126 54127 54128 54129 54130 54131 54132 54133 54134 54135
 54136 54137 54138 54139 54140 54141 54142 54143 54144 54145]

Date and time of shot 54143
('2018-12-18', '20:20:01')

Closest shot to 18:00, on 18.12.2018 - note, shot time is given and returned as float (hours)
(54137, 17.991388888888888)

Time of shot 54143, as float
20.3336111111


In [17]:
# Load the data from the the shot 54173
shot = 54173
tr = trace_at_shot(54173)

In [18]:
# plot the data

plot.plot_data(tr)

In [19]:
# plot only selected masses
plot.plot_data(tr, [18, 19, 20, 28])

In [21]:
# Define fitting parameters

# hydrogen-containing molecules
HM = {}

# HM['name of fitted gas'] = [pressure_parameter_name, name of molecule - has to match the name in the calibration
# definition of the isotope ratio]

# isotope ratio definition:
# 1: fitting_parameter_name
# 2: [fitting_parameter_name, lower boundary, upper boundary]
# 3: exact value

HM['hydrogen'] = ['h', 'hydrogen', 'ph']
HM['methane'] = ['m', 'methane', ['pm', 0, .1]]
HM['water'] = ['w', 'water', ['pw', .8, 1]]

# non-hydrogen containing molecules
NHM = {}
# [pressure_parameter_name, name of molecule in the calibration]
NHM['nitrogen'] = ['nit', 'N2']

# any masses that should be ignored in the fit are specified in disregard, in the following ways:
# 1 - mass1: ignore mass1
# [mass1, mass2, threshold]: ignore mass1 when mass2 >= threshold

# e.g.: disregard = [12, [14, 28, 1e-6]]
# ignore mass 12 always, ignore mass 14 when mass 28 is higher than 1e-6

disregard = []

# start time
t1 = -5
# end time
t2 = 30
# sample step size
# step = 1 - analyse every datapoint
# step = 3 - analyse every 3rd datapoint
step = 1

In [22]:
tr.deconvolute(HM, NHM, disregard, t1, t2, step)

Deconvolutiuon for 54173 done, duration 10.3663931027 s.


In [23]:
plot.show_results(tr)

C:\Anaconda2\lib\site-packages\matplotlib\cbook.py:137: MatplotlibDeprecationWarning: The set_color_cycle attribute was deprecated in version 1.5. Use set_prop_cycle instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [25]:
# plot only for water and methane
plot.show_results(tr, ['water', 'methane'])

In [ ]:
# the fitting results are stored in a dictionary called 'rescols' in the Trace object

# each fitted molecule has a 'pressure' column
# hydrogen containing molecules also have a 'ratio' column which contains the average H/(H+D) ratio

In [26]:
# evaluate the impact of uncertainty in the cracking patterns
# fit many times, each time with perturbed cracking patterns
# cracking patterns are perturbed randomly, with a given amplitude of perturbation

# amplidute of perturbation:
# 1 - as float (typical value = 0.1)
# 2 - as a dictionary, float for each of the molecules

perturb = {
    'hydrogen': 0,
    'water': 0.2,
    'methane': 0.1
}

times = 25
# to_join - joining of differnt isotopes into a single molecule
# for example ammonia with 15N and ammonia with 14N
# it is rarely used
# and not this time, so it is an empty argument:
to_join = [[], '']

trs = fmt.fit_many_times(tr, times, perturb, to_join, HM, NHM, disregard, -5, 30, 1)

Doing 25 for 54173 :
 | 1  | 2  | 3  | 4  | 5  | 6  | 7  | 8  | 9  | 10  | 11  | 12  | 13  | 14  | 15  | 16  | 17  | 18  | 19  | 20  | 21  | 22  | 23  | 24  | 25 deconvolution done, total duration 245.983759806 seconds


In [28]:
# get the average values from the fitted time traces
average_traces = fmt.povpreci(trs)
# and plot them, with errorbars
fmt.errorbar_results(average_traces)
plt.show()

In [ ]:
# data in average_traces is stored similarly to rescols
# each column (pressure and ratio) now consists of two subcolumns:
# val - average value of parameter
# std - standard deviation of parameter